In [2]:
import data
import ast_util
import ast
import evaluation

In [3]:
KEY = "tree"
dataset = data.Data("config.json", f"nagini_examples/{KEY}_finetune_full")
ev = evaluation.Evaluation(dataset)
examples = dataset.get_list_of_examples(KEY)
print(examples)
for example in examples:
    verified_program = dataset.get_example(KEY, example, "verified")
    verified_ast = ast.parse(verified_program, type_comments=True)
    print(ast.unparse(verified_ast))
    program_powerset = ast_util.dataset_generator(verified_ast)
    for i, program_ast in enumerate(program_powerset):
        if i == 0: continue
        program = ast.unparse(program_ast)
        verif_error = str(ev.verify_program_snippet(KEY, program, method_name=example))
        dataset.add_example_verified_ptr(f"{example}_{i}", program, verif_error, f"{example}_verified.txt", KEY)

['val_head', 'height', 'count', 'sum', 'insert', 'contains', 'inorder', 'min', 'mirror', 'subtree']
def val_at_head(node: TreeNode) -> int:
    """Returns the value at the head of the tree rooted at node"""
    Requires(tree(node))
    Ensures(tree(node))
    return Unfolding(tree(node), node.key)
response ['', 'Verification failed', 'Errors:', 'Assignment might fail. There might be insufficient permission to access tree(node). (tmp.py@38.4)', 'Verification took 11.66 seconds.']
response ['', 'Verification successful', 'Verification took 6.38 seconds.']
response ['', 'Verification failed', 'Errors:', 'Assignment might fail. There might be insufficient permission to access node.key. (tmp.py@39.4)', 'Verification took 5.97 seconds.']
response ['', 'Verification failed', 'Errors:', 'Assignment might fail. There might be insufficient permission to access tree(node). (tmp.py@37.4)', 'Verification took 5.87 seconds.']
response ['', 'Verification failed', 'Errors:', 'Assignment might fail. Th